In [26]:
#. 1 Import necessary libraries
import pennylane as qml 
from pennylane import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import pandas as pd


In [27]:
#. 2 Set up a Quantum device and define a Quantum Node

# Set up a quantum device with 4 wires (qubits)
dev = qml.device("default.qubit", wires=4) # This sets up a simulated quantum computer (called a device) with 4 qubits. This is where we run our quantum circuits.

# Define a simple quantum circuit (Quantum Node)

@qml.qnode(dev) # This decorator tells PennyLane that this function is a quantum circuit that can be executed on the quantum device.

def circuit(inputs, weights): # This function defines the quantum circuit. It takes inputs (the data) and weights (values we train) to produce an output.

    # Encode classical input into quantum state 
    for i in range(4):
        qml.RX(inputs[i], wires=i) # Each input value is turned into a rotation on a qubit. This encodes the classical data into quantum states.

    # Add entanglement and trainable parameters
    for i in range(4):
        qml.RY(weights[i], wires=i) # These are the "learnable" parts. We train the circuit by adjusting these rotation angles.

        return [qml.expval(qml.PauliZ(i)) for i in range(4)] # I measure each qubit and return values that tell us the final state of the quantum system. These values are used to make predictions.


In [28]:
#. 3 Select 4 Features from the dataset and scale them for Quantum input.

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv" # Loaded a dataset from a URL where the last column is the target variable (Outcome).
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
              'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'] # The dataset contains various health measurements and whether the person has diabetes (1) or not (0).
data = pd.read_csv(url, names=columns) # Read the dataset into a pandas DataFrame and assign column names.

# Select features and target variable
selected_features = ['Glucose', 'BMI', 'Age', 'Insulin'] # These are the features we will use for our quantum model. They are selected based on their relevance to diabetes prediction.
x = data[selected_features] # This extracts the selected features from the dataset.
y = data['Outcome'] # This is the target variable we want to predict (whether a person has diabetes or not).

# Scale the selected features between 0 and π (for use in quantum gates)
scaler = MinMaxScaler(feature_range=(0, np.pi)) # This scaler will adjust the feature values to fit within the range suitable for quantum gates.
x_scaled = scaler.fit_transform(x) # This applies the scaling to the selected features.



In [29]:
#. 4 Split the dataset into training and testing sets

# This splits the scaled data into training and testing sets. 80% of the data is used for training and 20% for testing.
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42) 

# Print the sizes for verification
print(f"Training set size: {len(x_train)}") # This prints the number of samples in the training set.
print(f"Testing set size: {len(x_test)}") # This prints the number of samples in the testing set.

Training set size: 614
Testing set size: 154


In [35]:
#. 5 Create a layer that wraps the quantum circuit

def quantum_layer(x_data, weights):
    results = []
    for x in x_data:
        result = circuit(x, weights) # This runs the quantum circuit for each input data point and collects the results.
        results.append(result)
    return np.array(results) # This returns the results as a NumPy array, which is easier to work with in Python.
                       
print("Quantum layer created.") # This confirms that the quantum layer has been successfully created.


Quantum layer created.


In [36]:
#. 6 Train a simple classifier using quantum outputs

np.random.seed(42) 
weights = np.random.uniform(0, np.pi, size=4) 

x_train_quantum = quantum_layer(x_train, weights) # This applies the quantum layer to the training data, producing quantum outputs.
x_test_quantum = quantum_layer(x_test, weights) # This applies the quantum layer to the testing data, producing quantum outputs.


# Train a classical model (Logistic Regression) on the quantum outputs
print("x_train_quantum shape:", x_train_quantum.shape)
print("y_train shape:", y_train.shape)


clf = LogisticRegression(max_iter=1000) # This creates a logistic regression model, which is a simple but effective classifier.
clf.fit(x_train_quantum, y_train) # This trains the logistic regression model using the quantum outputs from the training data.



x_train_quantum shape: (614, 4)
y_train shape: (614,)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'
